In [1]:
using Revise
using RDDIP
using Random
using Plots
using Gurobi
using Statistics
const GRB_ENV = Gurobi.Env()
optimizer=() -> Gurobi.Optimizer(GRB_ENV)

Set parameter Username
Set parameter LicenseID to value 2658456
Academic license - for non-commercial use only - expires 2026-04-28


#11 (generic function with 1 method)

In [2]:
T = 3
graph=RDDIP.LinearGraph(T);

In [6]:
function subproblem_builder(subproblem::Model, node::Int)
    # State variables
    H=[200, 200]
    @variable(subproblem, 0 <= volume[i=1:2] <= H[i], RDDIP.State, initial_value = 200)
    # Control variables
    @variables(subproblem, begin
        thermal_generation >= 0
        hydro_generation >= 0
        hydro_spill >= 0
    end)
    # Random variables
    @variable(subproblem, inflow)
    Ω = [0.0, 50.0, 100.0]
    P = [1 / 3, 1 / 3, 1 / 3]
    RDDIP.parameterize(subproblem, Ω, P) do ω
        return JuMP.fix(inflow, ω)
    end
    # Transition function and constraints
    @constraints(
        subproblem,
        begin
            [i = 1:2], volume[i].out == volume[i].in - hydro_generation - hydro_spill + inflow
            demand_constraint, hydro_generation + thermal_generation == 150
        end
    )
    # Stage-objective
    fuel_cost = [50, 100, 150]
    @stageobjective(subproblem, fuel_cost[node] * thermal_generation)
    return subproblem
end

subproblem_builder (generic function with 1 method)

In [7]:
model = RDDIP.PolicyGraph(
    subproblem_builder,
    graph;
    sense = :Min,
    lower_bound = 0.0,
    optimizer = optimizer,
)

A policy graph with 3 nodes.
 Node indices: 1, 2, 3


In [8]:
RDDIP.train(model; iteration_limit = 10)

-------------------------------------------------------------------
         RDDIP.jl (c) Oscar Dowson and contributors, 2017-25
-------------------------------------------------------------------
problem
  nodes           : 3
  state variables : 2
  scenarios       : 2.70000e+01
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : RDDIP.Expectation()
  sampling scheme : RDDIP.InSampleMonteCarlo
subproblem structure
  VariableRef                             : [9, 9]
  AffExpr in MOI.EqualTo{Float64}         : [3, 3]
  VariableRef in MOI.EqualTo{Float64}     : [1, 1]
  VariableRef in MOI.GreaterThan{Float64} : [6, 6]
  VariableRef in MOI.LessThan{Float64}    : [2, 3]
numerical stability report
  matrix range     [1e+00, 1e+00]
  objective range  [1e+00, 2e+02]
  bounds range     [2e+02, 2e+02]
  rhs range        [2e+02, 2e+02]
-------------------------------------------------------------------
 iteration    simulation      bound        time (s)     solve

In [2]:
instance=RDDIP.parse_nc4("Data/T-Ramp/10_0_1_w.nc4",  optimizer, 24);1

1

In [5]:
resSP=RDDIP.benders_callback(instance, RDDIP.extended_BD_FH_OH, silent=false, force=1.0, S=20, batch=1, gap=0.1, timelimit=120);

Set parameter Threads to value 1
Set parameter TimeLimit to value 1.1999999594688416e+02
Set parameter LazyConstraints to value 1
Set parameter MIPGap to value 0.001
Set parameter LazyConstraints to value 1
Set parameter MIPGap to value 0.001
Set parameter Threads to value 1
Set parameter TimeLimit to value 1.1999999594688416e+02
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: Intel(R) Core(TM) Ultra 9 185H, instruction set [SSE2|AVX|AVX2]
Thread count: 22 physical cores, 22 logical processors, using up to 1 threads

Non-default parameters:
TimeLimit  1.1999999594688416e+02
MIPGap  0.001
Threads  1
LazyConstraints  1

Optimize a model with 1947 rows, 2720 columns and 32893 nonzeros
Model fingerprint: 0x891bf16d
Variable types: 1990 continuous, 730 integer (730 binary)
Coefficient statistics:
  Matrix range     [5e-02, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve remove

In [ ]:
function subproblem_builder_UC(instance::Instance, force::Float64, subproblem::Model, node::Int)
    # State variables
    N=instance.N
    thermal_units=values(instance.Thermalunits)
    Next=instance.Next
    Buses=1:size(Next)[1]
    Lines=values(instance.Lines)
    BusWind=instance.BusWind
    NumWindfarms=length(BusWind)
    @variable(subproblem, thermal_units[i].MinPower <= power[i = 1:N] <= thermal_units[i].MaxPower, RDDIP.State, initial_value = thermal_units[i].MinPower)
    # Control variables
    @variables(subproblem, begin
        power_shedding >= 0
        power_curtailement >= 0
        is_on[i = 1:N, k = 1:2] >= 0 # k=1 current time, k=2 previous time
        start_up[i = 1:N] >= 0
        shut_down[i = 1:N] >= 0
        θ[b in Buses] 
        flow[b in Buses, bp in Next[b]]
    end)

    @constraint(model, [line in Lines, t in 1:T, s in 1:S], flow[line.b1,line.b2,t,s]<=line.Fmax)
    @constraint(model, [line in Lines, t in 1:T, s in 1:S], flow[line.b1,line.b2,t,s]>=-line.Fmax)
    @constraint(model, [line in Lines, t in 1:T, s in 1:S], flow[line.b1,line.b2,t,s]==line.B12*(θ[line.b1,t,s]-θ[line.b2,t,s]))

    # Random variables
    @variable(subproblem, error_forecast[b in Buses])
    Ω = [[0.0 for b in Buses]]
    P = [1.0]
    RDDIP.parameterize(subproblem, Ω, P) do ω
        for b in Buses
            JuMP.fix(error_forecast[b], ω[b])
        end
    end
    # Transition function and constraints
    @constraints(
        subproblem,
        begin
            [i = 1: N], power[i].out <= thermal_units[i].MaxPower * is_on[i, 1]
            [i = 1: N], power[i].out >= thermal_units[i].MinPower * is_on[i, 1]
            [i = 1: N], power[i].out - power[i].in <= (-thermal_units[i].DeltaRampUp)*start_up[i] + (thermal_units[i].MinPower + thermal_units[i].DeltaRampUp)*is_on[i, 1] - (thermal_units[i].MinPower)*is_on[i, 2]
            [i = 1: N], power[i].in - power[i].out <= (-thermal_units[i].DeltaRampDown)*shut_down[i] + (thermal_units[i].MinPower + thermal_units[i].DeltaRampDown)*is_on[i, 2] + (thermal_units[i].MinPower)*is_on[i, 1]
            [line in Lines], flow[line.b1,line.b2] <= line.Fmax
            [line in Lines], flow[line.b1,line.b2] >= -line.Fmax
            [line in Lines], flow[line.b1,line.b2] == line.B12*(θ[line.b1]-θ[line.b2])
            demand_constraint, [b in Buses], sum(power[unit.name].out for unit in thermal_units if unit.Bus==b) - power_curtailement[b] + power_shedding[b] == instance.Demandbus[b](1+force*error_forecast[b])+sum(flow[b,bp] for bp in Next[b])
        end
    )
    # Stage-objective
    @stageobjective(subproblem, sum(unit.LinearTerm*power[unit.name].out for unit in thermal_units)+sum(SHEDDING_COST*power_shedding[b]+CURTAILEMENT_COST*power_curtailement[b] for b in Buses))
    return subproblem
end

In [6]:
instance.N

10